In [3]:
from typing import *

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms

import cv2 as cv
import numpy as np

import matplotlib.pyplot as plt

from PIL import Image, ImageStat, ImageOps

from sot.bbox import BBox
from sot.utils import Size

plt.style.use('tableau-colorblind10')
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 11

%load_ext autoreload
%autoreload 2

In [5]:
def logistic_labels(x, y, r_pos, r_neg):
    dist = np.abs(x) + np.abs(y)  # block distance
    labels = np.where(dist <= r_pos,
                      np.ones_like(x),
                      np.where(dist < r_neg,
                               np.ones_like(x) * 0.5,
                               np.zeros_like(x)))
    return labels

r_pos, r_neg = 16, 0
total_stride = 8

size = 1, 1, 17, 17
# distances along x- and y-axis
n, c, h, w = size
x = np.arange(w) - (w - 1) / 2
y = np.arange(h) - (h - 1) / 2
x, y = np.meshgrid(x, y)

# create logistic labels
r_pos = r_pos / total_stride
r_neg = r_neg / total_stride
labels = logistic_labels(x, y, r_pos, r_neg)

labels

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0